<a href="https://colab.research.google.com/github/sridevibonthu/EVAsession2/blob/master/My_Copy_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import the required librarires
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
#Define the CNN Architecture
#torch.nn layers as which contain trainable parameters while torch.nn.functional are purely functional.
#The forward() pass defines the way we compute our output using the given layers and functions.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -28X28X1 OUtput - 28X28X32 RF - 3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input -28X28X32 OUtput - 28X28X64 RF - 5
        self.pool1 = nn.MaxPool2d(2, 2)  #input -28X28X64 OUtput - 14X14X64 RF - 6
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input -14X14X64 OUtput - 14X14X128 RF - 10
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #input -14X14X128 OUtput - 14X14X256 RF - 14
        self.pool2 = nn.MaxPool2d(2, 2) #input -14X14X256 OUtput - 7X7X256 RF - 16
        self.conv5 = nn.Conv2d(256, 512, 3)  #input -7X7X256 OUtput - 5X5X512 RF - 24
        self.conv6 = nn.Conv2d(512, 10, 5)  #input - 5X5X512 OUtput - 3X3X1024 RF - 32
       # self.conv7 = nn.Conv2d(1024, 10, 3)  #input -3X3X1024 OUtput - 1X1X10 RF - 40

    #Feed Forward network
    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        x = x.view(-1, 10)  #x reshaped from size [1X1X10] to [10],   Flatten
        return F.log_softmax(x)

In [0]:
!pip install torchsummary  #Model summary in PyTorch similar to model.summary() in Keras
from torchsummary import summary

#PyTorch can only perform operations on Tensors that are on the same device
#To run your network on the GPU, move the model and all necessary tensors to the GPU with .to(device) where device is either cuda or cpu
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)

summary(model, input_size=(1, 28, 28), ) #input_size is required to make a forward pass through the network.

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8             [-1, 10, 1, 1]         128,010
Total params: 1,696,010
Trainable params: 1,696,010
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.44
Params size (MB): 6.47
Estimated Total Size (MB): 7.91
----------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
#we could use num_workers > 1 to use subprocesses to asynchronously load data or 
#using pinned RAM (via pin_memory) to speed up RAM to GPU transfers. 
#these mostly matter when we're using a GPU.
#The values 0.1307 and 0.3081 used for the Normalize() transformation 
#below are the global mean and standard deviation of the MNIST dataset
torch.manual_seed(1)
batch_size = 128   #How many samples per batch to load?
#num_workers - number of subprocesses to use for data loading
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

#ToTensor() -convert data to torch.FloatTensor
#Prepare the Data loaders by choosing the training and test datasets
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() #we need to manually set the gradients to zero using optimizer.zero_grad() since PyTorch by default accumulates gradients. 
        output = model(data)
        loss = F.nll_loss(output, target) #compute a negative log-likelihodd loss between the output and the ground truth label
        loss.backward()   # The backward() call we now collect a new set of gradients which we propagate back into each of the network's parameters using optimizer.step()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
  #set all the layers in your model to evaluation mode. you should set your model to evaluation mode before testing or validating your model 
    model.eval()   
    test_loss = 0
    correct = 0
    # Using the context manager no_grad() we can avoid storing the computations done producing the output of our network in the computation graph.
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
#It is important to transfer the networks parameters to the appropriate device before passing them to the optimizer, 
#otherwise the optimizer will not be able to keep track of them in the right way.
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.5311, Accuracy: 7781/10000 (78%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.5198, Accuracy: 7803/10000 (78%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.5081, Accuracy: 7841/10000 (78%)



loss=0.457878440618515 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 37.72it/s]



Test set: Average loss: 0.5046, Accuracy: 7841/10000 (78%)

